In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, median_absolute_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
file_path = "train_FD004_processed.csv"
df = pd.read_csv(file_path)

if df.isna().any().any():
    print("NaN values found in dataset. Removing rows with NaN.")
    df = df.dropna()

max_rul = 130 
df['RUL'] = df['RUL'].clip(upper=max_rul)

WINDOW_SIZE = 30
FEATURE_COLS = [col for col in df.columns if col.startswith('op_setting_') or col.startswith('sensor_measurement_')]
TARGET_COL = 'RUL'

In [4]:
variances = df[FEATURE_COLS].var()
selected_features = variances[variances > 0.01].index.tolist()  # Keep features with variance > 0.01
print(f"Selected features: {selected_features}")
df = df[['unit_number'] + selected_features + [TARGET_COL]]

Selected features: ['op_setting_1', 'op_setting_2', 'op_setting_3', 'sensor_measurement_1', 'sensor_measurement_2', 'sensor_measurement_3', 'sensor_measurement_4', 'sensor_measurement_5', 'sensor_measurement_6', 'sensor_measurement_7', 'sensor_measurement_8', 'sensor_measurement_9', 'sensor_measurement_10', 'sensor_measurement_11', 'sensor_measurement_12', 'sensor_measurement_13', 'sensor_measurement_14', 'sensor_measurement_15', 'sensor_measurement_16', 'sensor_measurement_17', 'sensor_measurement_18', 'sensor_measurement_19', 'sensor_measurement_20', 'sensor_measurement_21']


In [5]:
scaler_features = StandardScaler()
scaler_target = StandardScaler()

df[selected_features] = scaler_features.fit_transform(df[selected_features])

df[TARGET_COL] = scaler_target.fit_transform(df[[TARGET_COL]])

if df.isna().any().any():
    print("NaN values found after normalization. Removing rows with NaN.")
    df = df.dropna()


In [6]:
def create_sequences(data, window_size, feature_cols, target_col):
    X, y = [], []
    for unit in data['unit_number'].unique():
        unit_data = data[data['unit_number'] == unit]
        feature_data = unit_data[feature_cols].values
        target_data = unit_data[target_col].values
        for i in range(len(unit_data) - window_size):
            X.append(feature_data[i:i+window_size])
            y.append(target_data[i+window_size])
    return np.array(X), np.array(y)

In [7]:
X, y = create_sequences(df, WINDOW_SIZE, selected_features, TARGET_COL)

In [8]:
if np.any(np.isnan(X)) or np.any(np.isnan(y)):
    print("NaN values found in sequences. Removing affected samples.")
    mask = ~np.isnan(X).any(axis=(1, 2)) & ~np.isnan(y)
    X, y = X[mask], y[mask]

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
input_shape = X.shape[1:] 
inputs = Input(shape=input_shape)

In [10]:
x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.2)(x)
x = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.2)(x)
x = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.2)(x)

In [11]:
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(1)(x)

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005, clipnorm=1.0)  # Reduced learning rate and added gradient clipping
model = Model(inputs, outputs)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30, 24)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 30, 64)         │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 15, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 15, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 15, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 15, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 7, 256)         │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 256)         │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 3, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 3, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 236,481 (923.75 KB)

 Trainable params: 235,585 (920.25 KB)

 Non-trainable params: 896 (3.50 KB)

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

In [14]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, lr_scheduler],
    verbose=1
)

Epoch 1/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 39s 23ms/step - loss: 0.8281 - mae: 0.7074 - val_loss: 0.3320 - val_mae: 0.4897 - learning_rate: 5.0000e-04
Epoch 2/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 29s 21ms/step - loss: 0.3412 - mae: 0.4580 - val_loss: 0.2897 - val_mae: 0.4482 - learning_rate: 5.0000e-04
Epoch 3/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 27s 20ms/step - loss: 0.3153 - mae: 0.4402 - val_loss: 0.2672 - val_mae: 0.4316 - learning_rate: 5.0000e-04
Epoch 4/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 44s 22ms/step - loss: 0.3020 - mae: 0.4284 - val_loss: 0.2599 - val_mae: 0.4199 - learning_rate: 5.0000e-04
Epoch 5/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 29s 21ms/step - loss: 0.2885 - mae: 0.4193 - val_loss: 0.2487 - val_mae: 0.4198 - learning_rate: 5.0000e-04
Epoch 6/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 30s 22ms/step - loss: 0.2672 - mae: 0.4001 - val_loss: 0.2387 - val_mae: 0.4100 - learning_rate: 5.0000e-04
Epoch 7/50
1345/1345 ━━━━━━━━━━━━━━━━━━━━ 29s 21ms/step - loss: 0.2539 - mae: 0.3911 - val_loss: 0.2

In [15]:
y_pred = model.predict(X_val)

337/337 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [16]:
if np.any(np.isnan(y_pred)):
    print("NaN values found in predictions. Replacing with 0.")
    y_pred = np.nan_to_num(y_pred, 0)

In [17]:
y_val_inv = scaler_target.inverse_transform(y_val.reshape(-1, 1)).flatten()
y_pred_inv = scaler_target.inverse_transform(y_pred).flatten()

In [18]:
if np.any(np.isnan(y_val_inv)) or np.any(np.isnan(y_pred_inv)):
    print("NaN values found after inverse transformation. Removing affected samples.")
    mask = ~np.isnan(y_val_inv) & ~np.isnan(y_pred_inv)
    y_val_inv, y_pred_inv = y_val_inv[mask], y_pred_inv[mask]

In [19]:
mse = mean_squared_error(y_val_inv, y_pred_inv)
mae = mean_absolute_error(y_val_inv, y_pred_inv)
rmse = np.sqrt(mse)
r2 = r2_score(y_val_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_val_inv, y_pred_inv)
medae = median_absolute_error(y_val_inv, y_pred_inv)

In [20]:
print(f"MSE: {mse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"Median Absolute Error: {medae:.4f}")

MSE: 97.3889
MAE: 7.3907
RMSE: 9.8686
R² Score: 0.9478
MAPE: 4668151777395.25%
Median Absolute Error: 5.0792


In [27]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('training_loss_plot.png')
plt.close()

In [22]:
plt.figure(figsize=(10, 6))
plt.scatter(y_val_inv, y_pred_inv, alpha=0.5, color='blue')
plt.plot([y_val_inv.min(), y_val_inv.max()], [y_val_inv.min(), y_val_inv.max()], 'r--', lw=2, label='Ideal Fit')
plt.title('Predicted vs Actual RUL')
plt.xlabel('Actual RUL')
plt.ylabel('Predicted RUL')
plt.legend()
plt.grid(True)
plt.savefig('predicted_vs_actual_rul.png')
plt.close()

In [23]:
feature_importance = pd.Series(variances.loc[selected_features], index=selected_features)
plt.figure(figsize=(12, 6))
feature_importance.sort_values(ascending=False).plot(kind='bar')
plt.title('Feature Importance Based on Variance')
plt.xlabel('Features')
plt.ylabel('Variance')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('feature_importance_plot.png')
plt.close()

In [24]:
model.save("fixed_enhanced_cnn_rul_model.keras")